<a href="https://colab.research.google.com/github/saykim/ds/blob/main/keras_cnn_gpt_gridsearchCV_230222.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) 기본 모델

In [ ]:
# 필요한 라이브러리를 import 합니다
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 데이터 경로 설정
data_path = '/path/to/data'
train_path = os.path.join(data_path, 'train')  # train 데이터 경로
test_path = os.path.join(data_path, 'test')    # test 데이터 경로

# train 폴더의 파일 이름과 라벨 추출
filenames = os.listdir(train_path)
labels = [1 if 'abnormal' in filename else 0 for filename in filenames]

# 파일 이름과 라벨 정보를 포함한 데이터프레임 생성
df = pd.DataFrame({'filename': filenames, 'label': labels})
df.to_csv(os.path.join(data_path, 'labels.csv'), index=False)

# 데이터 로드 및 전처리
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# flow_from_directory 함수를 사용하여 train_data와 test_data 생성
train_data = train_datagen.flow_from_directory(train_path, target_size=(224, 224), batch_size=32, class_mode='categorical', classes=['normal', 'abnormal'])
test_data = test_datagen.flow_from_directory(test_path, target_size=(224, 224), batch_size=32, class_mode='categorical', classes=['normal', 'abnormal'])

# 모델 구성
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(train_data, epochs=10, validation_data=test_data)

# 테스트 데이터 예측
pred = model.predict(test_data)
labels = pred.argmax(axis=1)

# 파일 이름에 따른 라벨 출력
for i, label in enumerate(labels):
    print(test_data.filenames[i], label)


# 2) 이미지 변형 추가

## 데이터 경로 설정

In [ ]:
import os

data_path = '/path/to/data'
train_path = os.path.join(data_path, 'train')
test_path = os.path.join(data_path, 'test')


## 데이터 로드 및 전처리

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

train_data = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_data = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


1. ImageDataGenerator 클래스를 사용하여 데이터를 로드하고 전처리합니다. train_datagen 객체에는 학습 데이터를 위한 이미지 전처리 방법을 지정합니다. 여기에서는 이미지 회전, 가로/세로 이동, 축소/확대, 수평/수직 뒤집기 등의 변형을 지정합니다.

2. test_datagen 객체는 테스트 데이터에 대한 전처리를 수행합니다. 여기에서는 이미지의 크기만 조정합니다.

3. flow_from_directory 메서드를 사용하여 디렉토리에서 이미지를 로드합니다. target_size는 이미지를 리사이징할 크기를 지정합니다. batch_size는 배치 크기를 지정합니다. class_mode는 이진 분류일 경우 'binary', 다중 분류일 경우 'categorical'로 설정합니다.

## 모델 구성

* 모델을 구성하는 코드는 다음과 같습니다. 
> input_layer를 Input 클래스의 인스턴스로 지정하고, 이후에 Conv2D, MaxPooling2D, Flatten, Dense, Dropout 등의 레이어를 쌓아서 모델을 구성합니다.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

input_layer = Input(shape=(224, 224, 3))

x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)

output_layer = Dense(1, activation='sigmoid')(x)

model = Model(input_layer, output_layer)


## 모델 컴파일

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


## 모델 학습

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 함수
earlystop_callback = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(train_data, epochs=10, validation_data=test_data, callbacks=[earlystop_callback])


## 모델 평가

In [ ]:
score = model.evaluate(test_data)
print('Test accuracy:', score[1])


### 학습 예측결과 다르게 나온 이미지 만 확인

In [ ]:
predicted_labels = model.predict(test_data) # 수정 필요
true_labels = test_data.classes

# 예측 결과와 실제 라벨 간 비교
misclassified_indices = [i for i in range(len(predicted_labels)) if predicted_labels[i] != true_labels[i]]

# 예측이 틀린 이미지만을 선택하여 추출
misclassified_images = []
for i in misclassified_indices:
    x, y = test_data[i]
    misclassified_images.append(x)

# 예측이 틀린 이미지 출력
import matplotlib.pyplot as plt

for img in misclassified_images:
    plt.imshow(img)
    plt.show()


## 모델 예측

In [ ]:
pred = model.predict(test_data)


## 모델 저장

In [ ]:
model.save('my_model.h5')


## 모델 로드

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('my_model.h5')


## 모델 예측

In [ ]:
pred = model.predict(test_data)


## 모델 평가

In [ ]:
score = model.evaluate(test_data)
print('Test accuracy:', score[1])


## 모델 시각화

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)


# GridSeachCV, 적용한 Keras 모델

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.optimizers import Adam

# 모델 생성 함수
def create_model(filters, kernel_size, learning_rate):
    inputs = Input(shape=(224, 224, 3))
    x = Conv2D(filters, kernel_size, activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(filters, kernel_size, activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(2, activation='softmax')(x)

    model = Model(inputs, outputs)

    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# 데이터 경로 설정
data_path = '/path/to/data'
train_path = os.path.join(data_path, 'train')
test_path = os.path.join(data_path, 'test')

# 데이터 로드 및 전처리
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_path, target_size=(224, 224), batch_size=32,
                                               class_mode='categorical', classes=['normal', 'abnormal'])
test_data = test_datagen.flow_from_directory(test_path, target_size=(224, 224), batch_size=32,
                                             class_mode='categorical', classes=['normal', 'abnormal'])

# 모델 래핑
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# 하이퍼파라미터 그리드
param_grid = {'filters': [32, 64, 128],
              'kernel_size': [(3, 3), (5, 5), (7, 7)],
              'learning_rate': [0.001, 0.0001, 0.00001]}

# 그리드 탐색
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_data)

# 결과 출력
print(f'Best score: {grid_result.best_score_} using {grid_result.best_params_}')


## 라벨명 파일 추가 

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.optimizers import Adam

# 모델 생성 함수
def create_model(labels, filters, kernel_size, learning_rate):
    inputs = Input(shape=(224, 224, 3))
    x = Conv2D(filters, kernel_size, activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(filters, kernel_size, activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(2, activation='softmax')(x)

    model = Model(inputs, outputs)

    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# 데이터 경로 설정
data_path = '/path/to/data'
train_path = os.path.join(data_path, 'train')
test_path = os.path.join(data_path, 'test')

# 데이터 로드 및 전처리
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_path, target_size=(224, 224), batch_size=32,
                                               class_mode='categorical', classes=['normal', 'abnormal'])
test_data = test_datagen.flow_from_directory(test_path, target_size=(224, 224), batch_size=32,
                                             class_mode='categorical', classes=['normal', 'abnormal'])

# 라벨 파일 생성
labels = {v: k for k, v in train_data.class_indices.items()}

# 모델 래핑
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0, labels=labels)

# 하이퍼파라미터 그리드
param_grid = {'filters': [32, 64, 128],
              'kernel_size': [(3, 3), (5, 5), (7, 7)],
              'learning_rate': [0.001, 0.0001, 0.00001]}

# 그리드 탐색
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_data)

# 결과 출력
print(f'Best score: {grid_result.best_score_} using {grid_result.best_params_}')


> 위 코드에서는 create_model 함수에서 라벨 파일을 labels 인자로 받도록 수정했습니다. create_model 함수에 labels 인자가 추가되었기 때문에, 이제 모델 래핑 시 labels 인자를 추가하여 labels 값을 전달해야 합니다. labels 값은 train_data.class_indices 딕셔너리의 key와 value를 뒤집은 형태로 생성하였습니다.

* 그리드 탐색 이후에는, 최적의 모델과 하이퍼파라미터로 최종 모델을 학습하고, 테스트 데이터에 대한 성능을 평가할 수 있습니다. 이를 위해서는 다음과 같은 코드를 사용할 수 있습니다.

In [ ]:
from tensorflow.keras.optimizers import Adam

# 최적의 하이퍼파라미터로 모델 생성
best_model = create_model(labels, filters=32, kernel_size=(3, 3), learning_rate=0.001)

# 모델 학습
optimizer = Adam(lr=0.001)
best_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
best_model.fit(train_data, epochs=10, batch_size=32, validation_data=test_data, verbose=1, callbacks=[early_stopping])

# 모델 평가
score = best_model.evaluate(test_data, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')


> 위 코드에서는 create_model 함수에 labels 인자를 전달하여 최적의 모델을 생성하고, 최적의 하이퍼파라미터로 모델을 학습시킵니다. 이후에는 evaluate 메서드를 사용하여 테스트 데이터에 대한 성능을 평가합니다